In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from easygui import *
import plotly.io as pio

In [2]:
#df is your players' data, just change the path to your data file's path.
df = pd.read_html(r"/Users/Zhaoyichun/Downloads/Untitled.html",skiprows=0,)[0]
df.head()


,Tgls/90,Name,Age,Personality,Best Pos,Expires,Transfer Value,Wage,Svt,Svp,...,Shot/90,Aer A/90,xG,Height,Cons,FA,Fls,Yel,Gwin,Clear
0,2.95,Benjamin Å eÅ¡ko,21,Balanced,ST (C),30/6/2029,â¬102M - â¬138M,"â¬100,000 p/w",-,-,...,4.85,11.07,29.30,194 cm,12,111,48,0,92%,45.00
1,2.80,Karim Adeyemi,22,Driven,AM (R),30/6/2028,â¬107M - â¬135M,"â¬100,000 p/w",-,-,...,3.76,8.46,17.06,179 cm,13,95,70,1,90%,41.00
2,2.68,Lorenzo Lucca,23,Driven,ST (C),30/6/2027,â¬63M - â¬77M,"â¬38,000 p/w",-,-,...,5.23,12.96,13.36,201 cm,14,52,14,2,94%,32.00
3,2.74,Dominik Szoboszlai,23,Balanced,AM (L),30/6/2027,â¬118M - â¬129M,"â¬80,000 p/w",-,-,...,2.12,9.58,13.59,186 cm,14,32,47,9,94%,185.00
4,2.74,Cole Palmer,22,Balanced,AM (R),30/6/2028,â¬83M - â¬111M,"â¬70,000 p/w",-,-,...,3.44,9.80,10.52,187 cm,10,97,56,6,90%,57.00


In [3]:
# Get rid of symbols or something not good for analyzing
pd.set_option('display.max_columns',40)
pd.set_option('display.max_rows',30)
for column in ['Svt', 'Svh', 'Svp', 'Ch C/90', 'DrbPG', 'Cr C', 'Cr C/A', 'Tck', 'Int/90', 'K Ps/90', 'Hdrs W/90',
               'Hdr %','Ps C/90', 'Pas %','Gls/90', 'Asts/90', 'ShT/90', 'Tck R', 'Gls', 'Shot %', 'Tcon/90',
               'Ps A/90', 'Mins', 'Dist/90', 'Av Rat', 'Ast', 'Tcon', 'Sv %', 'Shot/90', 'Aer A/90', 'xG', 'Height'
              ]:
    df[column] = df[column].apply(str).str.replace('-','0')
    df[column] = df[column].apply(str).str.replace('%','')
    df[column] = df[column].apply(str).str.replace('cm','')
    df[column] = df[column].apply(str).str.replace('km','').astype(float)

    df['xG'] = df['xG'].apply(str).str.replace('0.0','0.01').astype(float)
    df['Wage'] = df['Wage'].apply(str).str.replace('p/w','')
    df['Wage'] = df['Wage'].apply(str).str.replace(',','')
    df['Wage'] = df['Wage'].apply(str).str.replace('â\x82¬','').astype(float)
    df['Transfer Value'] = df['Transfer Value'].apply(str).str.replace('â\x82¬','')
    df['Transfer Value'] = df['Transfer Value'].apply(str).str.replace('M','0')

/var/folders/js/l945ghm905b7hw9ky4qvhp0r0000gn/T/ipykernel_734/1297237242.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['xG'] = df['xG'].apply(str).str.replace('0.0','0.01').astype(float)


In [35]:
# Define more types of stat
df['Tot_Sav'] = df['Svp'] + df['Svh'] + df['Svt']
df['Per_90'] = df['Mins'] / 90
df['Saves_90'] = df['Tot_Sav'] / df['Per_90']
df['Shots_Faced'] = df['Tcon'] + df['Tot_Sav']
df['Faced_90'] = df['Shots_Faced'] / df['Per_90']
df['Goal'] = df["Gls"] / df["xG"]
df['AttOutput'] = df['Gls/90'] + df['Asts/90']
df['ShTratio'] = df['ShT/90'] / df['Shot/90']
df['ShTratio'] = df['ShTratio'].fillna(0)
df['XG-Goal'] = df['Gls'] - df['xG'].astype(float)

In [24]:
#Analysing Goalkeepers
condition = (df['Best Pos']=='GK')
included = df[condition]
fig = px.scatter(included, x="Faced_90", y="Saves_90", text="Name", log_x=False, size_max=40, size='Sv %')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Shots faced/90')
fig.update_yaxes(title_text='Saves made/90')
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Shot Stopping")
fig.show()


In [72]:
#Analysing Defending Quality
fig = px.scatter(df, x="Tck", y="Hdrs W/90", text="Name", log_x=False, size_max=15, color="Ps C/90" , size='Tck R')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Tackles Won/90')
fig.update_yaxes(title_text='Headers Won/90')
fig.add_hline(y=6, opacity=1, line_width=2, line_dash='dash', line_color='green')
fig.add_vline(x=0.94, opacity=1, line_width=2, line_dash='dash', line_color='green')
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Defending Quality")
fig.show()

In [85]:
#Analysing Ground Duels ability
fig = px.scatter(df, x="Tck", y="Int/90", text="Name", log_x=False, size_max=20, color="Tck R", size= 'Fls')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Tackles Won/90')
fig.update_yaxes(title_text='Interceptions/90')
fig.add_hline(y=1.77, opacity=1, line_width=2, line_dash='dash', line_color='green')
fig.add_hrect(y0=1.77, y1=df["Int/90"].max()+0.4, line_width=0, fillcolor="green", opacity=0.2)
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Ground Duels")
fig.show()

In [83]:
#Analysing Aerial Duels ability
fig = px.scatter(df, x="Hdrs W/90", y="Hdr %", text="Name", log_x=False, size_max=60, color="Height")
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Headers Won/90')
fig.update_yaxes(title_text='Headers Won %')
fig.add_hline(y=80, opacity=1, line_width=2, line_dash='dash', line_color='green')
fig.add_hrect(y0=80, y1=100, line_width=0, fillcolor="green", opacity=0.2)
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Aerial Duels")
fig.show()

In [82]:
#Analysing Ball Carrying ability
fig = px.scatter(df, x="Ch C/90", y="DrbPG", text="Name", log_x=False, size_max=30, color='FA')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Progressive Passes/90')
fig.update_yaxes(title_text='Progressive Runs/90')
fig.add_hline(y=1.71, opacity=1, line_width=2, line_dash='dash', line_color='green')
fig.add_hrect(y0=1.71, y1=df["DrbPG"].max()+0.4, line_width=0, fillcolor="green", opacity=0.2)
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Ball carrying skills")
fig.show()

In [12]:
#Analysing Crossing ability
fig = px.scatter(df, x="Cr C", y="Cr C/A", text="Name", log_x=False, size_max=20, color='Asts/90',size="Ast")
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Crossed Completed/90')
fig.update_yaxes(title_text='Crossing Accuracy')
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Crossing ability")
fig.show()

In [67]:
#Analysing Chance Creation ability
fig = px.scatter(df, x="K Ps/90", y="Ch C/90", text="Name", log_x=False, size_max=20, color='Asts/90',size="Ast")
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Chance Creation/90')
fig.update_yaxes(title_text='Key Passes/90')
fig.add_vline(x=0.91, opacity=1, line_width=2, line_dash='dash', line_color='green')
fig.add_vrect(x0=0.91, x1=df["K Ps/90"].max()+0.4, line_width=0, fillcolor="green", opacity=0.2)
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Chance Creation")
fig.show()

In [61]:
#Analysing Passing ability
fig = px.scatter(df, x="Ps C/90", y="Pas %", text="Name", log_x=False, size_max=20, color='K Ps/90',size= 'K Ps/90')
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Passes Completed/90')
fig.update_yaxes(title_text='Pass Completion %')
fig.add_hline(y=83, opacity=1, line_width=2, line_dash='dash', line_color='green')
fig.add_hrect(y0=83, y1=100, line_width=0, fillcolor="green", opacity=0.2)
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Passing Ability")
fig.show()

In [60]:
#Analysing Attacking Impact for the team
fig = px.scatter(df, x="Gls/90", y="Asts/90", text="Name", log_x=False, size_max=20, color ='Av Rat', size= 'AttOutput')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Goals/90')
fig.update_yaxes(title_text='Assists/90')
fig.add_hline(y=0.2, opacity=1, line_width=2, line_dash='dash', line_color='green')
fig.add_hrect(y0=0.2, y1=df["Asts/90"].max()+0.1, line_width=0, fillcolor="green", opacity=0.2)
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Attacking Impact")
fig.show()

In [58]:
#Analysing Goalscoring Efficiency for the team
fig = px.scatter(df, x="Gls/90", y="ShT/90", text="Name", log_x=False, size_max=20, color='Shot/90', size = 'ShTratio')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Goals/90')
fig.update_yaxes(title_text='Shots on Target/90')
fig.add_vline(x=0.39, opacity=1, line_width=2, line_dash='dash', line_color='green')
fig.add_vrect(x0=0.39, x1= df["Gls/90"].max() + 0.1, line_width=0, fillcolor="green", opacity=0.2)
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Goalscoring Efficiency")
fig.show()

In [57]:
#See the relation between goals and xG
fig = px.scatter(df, x="Gls", y="xG", text="Name", log_x=False, size_max=15, color='ShT/90', size='Gls')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Goals')
fig.update_yaxes(title_text='Expected Goals')

for template in ["seaborn"]:
    fig.update_layout(template=template, title="Goalscoring Expectation VS Goals")
fig.show()

In [44]:
#Analysing xG and Goals to see who outperform in the team
fig = px.scatter(df, x='XG-Goal', y="xG", text="Name", log_x=False, size_max=15, color='ShT/90', size='Gls')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Actual Goals min xG')
fig.update_yaxes(title_text='Expected Goals')

for template in ["seaborn"]:
    fig.update_layout(template=template, title="Goals beyond expectation")
fig.show()

In [46]:
#Running distance per game and minutes played for each player
fig = px.scatter(df, x="Dist/90", y="Mins", text="Name", log_x=False, size_max=100, color='Av Rat')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Distance covered/90')
fig.update_yaxes(title_text='Minitues')
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Running Distance")
fig.show()

In [56]:
#Analysing players' rating to their hidden ability(consistency)

total = sum(df['Av Rat'])
avgRating = total / df.shape[0]

fig = px.scatter(df, x="Av Rat", y="Cons", text="Name", log_x=False, size_max=20, color='Av Rat',size = 'Cons')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Average Rating')
fig.update_yaxes(title_text='Consistency')
fig.add_vline(x=avgRating, opacity=1, line_width=2, line_dash='dash', line_color='green')
fig.add_vrect(x0=avgRating, x1= df["Av Rat"].max() + 0.2, line_width=0, fillcolor="green", opacity=0.2)
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Players' States")
fig.show()

In [80]:
#Analysing fouls and yellow cards

fig = px.scatter(df, x="Fls", y="Yel", text="Name", log_x=False, size_max=20, color='Av Rat',size = 'Cons')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Fouls comitted')
fig.update_yaxes(title_text='Yellow cards')
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Players' Diciplines of season")
fig.show()

In [96]:
#The relation between wage and performance 
fig = px.scatter(df, x="Wage", y="Av Rat", text="Name", log_x=False, size_max=20, color='Av Rat')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Wage p/w')
fig.update_yaxes(title_text='Ave Rating')
for template in ["seaborn"]:
    fig.update_layout(template=template, title="No effort players analysis")
fig.show()

In [130]:
#The relation between player's transfer value and performance 

df['Average TransferValue'] = (df['Transfer Value'].str.split('-').str.get(0).astype(float) + df['Transfer Value'].str.split('-').str.get(1).astype(float)) / 2
df['Average TransferValue']

fig = px.scatter(df, x='Average TransferValue', y="Av Rat", text="Name", log_x=False, size_max=20, color='Av Rat')
#fig.update_yaxes(autorange="reversed")
fig.update_traces(textposition='top center')
fig.update_layout(height=500)
fig.update_xaxes(title_text='Transfer Value (M/€）')
fig.update_yaxes(title_text='Ave Rating')
for template in ["seaborn"]:
    fig.update_layout(template=template, title="Players' value")
fig.show()